https://radimrehurek.com/gensim/models/ldamodel.html

https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/

# Configuration

In [5]:
import os
import sys
rootpath = os.path.dirname(os.getcwd())
sys.path.append(rootpath)

from news import LdaGridSearchResult, NewsIO, NewsPath
newsio = NewsIO()
newspath = NewsPath()

import json
import pandas as pd
from collections import defaultdict

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
## Parameters
SAMPLE_SIZE = 100000
MAX_WORD_TOPIC = 50

## Filenames
fname_gs_result = f'lda_gs_{SAMPLE_SIZE}.json'

fname_docs_dict = f'lda/docs_dict_{SAMPLE_SIZE}.json'
fname_id2word = f'lda/id2word_{SAMPLE_SIZE}.json'
fname_docs_bow = f'lda/docs_bow_{SAMPLE_SIZE}.json'

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def find_optimum(lda_gs_result):
    result_list = [(fname, coherence) for fname, coherence in zip(lda_gs_result.result['fname'], lda_gs_result.result['coherence'])]
    result_list_sorted = list(sorted(result_list, key=lambda x:x[1], reverse=True))
    return result_list_sorted[0]

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
## Find optimum
gs_result = newsio.load(_type='result', fname_object=fname_gs_result, verbose=False)
lda_gs_result = LdaGridSearchResult(gs_result=gs_result)

fname_lda_opt, coherence_opt = find_optimum(lda_gs_result)
lda_model = newsio.load(_type='model', fname_object=fname_lda_opt)

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  | fdir : /data/blank54/workspace/project/news/model
  | fname: lda/lda_100000_37_10_0.1_0.02.pk


# LDA model inference

In [5]:
docs_dict = newsio.load(_type='data', fname_object=fname_docs_dict)
id2word = newsio.load(_type='data', fname_object=fname_id2word)
docs_bow = newsio.load(_type='data', fname_object=fname_docs_bow)

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  | fdir : /data/blank54/workspace/project/news/data
  | fname: lda/docs_dict_100000.json
  | fdir : /data/blank54/workspace/project/news/data
  | fname: lda/id2word_100000.json
  | fdir : /data/blank54/workspace/project/news/data
  | fname: lda/docs_bow_100000.json


In [5]:
topics = lda_model.show_topics(num_topics=lda_model.num_topics, num_words=MAX_WORD_TOPIC, formatted=False)

topic_keywords = defaultdict(list)
for topic_id, word_list in topics:
    topic_keywords['topic_id'].append(topic_id)
    for idx, (word, score) in enumerate(word_list):
        topic_keywords[f'word_{idx}'].append(word)
        
topic_keywords_df = pd.DataFrame(topic_keywords)

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
fname_topic_keywords = 'topic_keywords.xlsx'
topic_keywords_df.to_excel(excel_writer=os.path.sep.join((newspath.fdir_result, fname_topic_keywords)))

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Topic assignment

In [6]:
from news import NewsCorpus
import numpy as np
import itertools

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
corpus = NewsCorpus()

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
for doc in corpus.iter():
    print(doc['content'])
    print()
    print(doc['nouns_stop'])
    doc_bow = id2word.doc2bow(itertools.chain(*doc['nouns_stop']))
    print(doc_bow)
    inf = lda_model.inference([doc_bow])
    break

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|          | 0/2358757 [00:00<?, ?it/s]Exception ignored in: <generator object NewsCorpus.iter at 0x7f3f84f8d4d0>
RuntimeError: generator ignored GeneratorExit
  0%|          | 2/2358757 [00:00<2:05:20, 313.64it/s]

ArticleReadingError: /data/blank54/workspace/project/news/corpus/201207/a-0002424076.json
안택수 신용보증기금 이사장(왼쪽)이 12일 서울 도화동 가든호텔에서 마지막 기자간담회를 열어 금융당국이 신보의 부실채권을 일방적으로 자산관리공사에 이관하는 것에 반대 한다는 입장을 분명히 하고 있다.안택수 신용보증기금(신보) 이사장은 12일 "부실채권을 기술보증기금처럼 자산관리공사(캠코)에 무조건 넘기는 것은 용납할 수 없는 일"이라고 말했다. 그는 "금융위원회가 그렇게 하고 싶다면 신보가 거둬들인 회수율만큼 대가를 지급하고 가져가라"고 반대 입장을 분명히 했다.오는 17일 퇴임하는 안 이사장은 이날 마지막 기자간담회를 열어 그동안의 성과와 아쉬운 점 등에 대해 가감없이 입장을 밝혔다. 우선 그는 최근 금융위가 신보의 부실채권을 캠코로 이관하려는 것과 관련, "신보는 4년째 연간 7000억원 수준의 부실채권을 회수해 수익률이 20%를 기록하고 있다"면서 "하지만 이를 캠코로 넘기면 수익률이 10%밖에 되지 않는다"고 말했다.올해 캠코는 총 4073억원 규모의 중소기업진흥공단 부실채권을 인수한 데 이어 하반기에 기술보증기금이 보유한 약 5조원 규모의 부실채권을 추가로 인수할 계획이다. 아울러 신보법을 개정, 신보가 보유한 부실채권도 캠코로 이관하는 방안을 추진하고 있다.신보 노용훈 관리부 본부장은 "신보는 지난 36년간 부실채권 회수를 담당하는 직원을 별도로 두고 부실채권 관리시스템을 구축하는 등 이 분야에서 노하우를 가지고 있다"면서 "현재 정부와 협의 중인데 5년이 지났다고 무조건 매각하는 것은 아니고 채무자들이 도덕적 해이(모럴해저드)에 빠지지 않도록 단계적으로 매각하는 방안을 캠코와 논의하고 있다"고 밝혔다. 이와 함께 안 이사장은 "올해 하반기에 경기침체기로 들어가면 보증 총량을 40조4000억원(보증잔액 기준)까지 늘릴 것"이라며 "경기침체기에 들어설 경우 보증수수료도 낮춰야 할 것"이라고 말

In [11]:
inf

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(array([[ 0.10001215,  0.10000842,  0.10002447,  0.10003351,  0.10002286,
          0.10000772, 14.291891  ,  0.10001052,  0.10000343,  0.10002243,
          0.10001021,  0.10001378,  1.5785474 ,  0.10000712,  0.10000902,
          0.10001113,  0.10002867,  0.10001866,  0.10000719,  0.1000089 ,
          0.10000721,  0.10000953,  0.1000098 ,  2.2791893 ,  0.10003704,
          0.10001487,  0.10001907,  0.33722547,  0.10003563,  2.9941823 ,
          0.10001881,  0.10000343,  0.10001417,  0.10000748,  0.10001086,
          0.10002864,  0.10001975]], dtype=float32),
 None)

In [16]:
np.argmax(inf[0])

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


6